In [ ]:
# | default_exp _helpers.utils


In [ ]:
# | export


import os
from typing import *
from contextlib import contextmanager
from pathlib import Path

import nbdev
from configparser import ConfigParser


In [ ]:
import shutil
from tempfile import TemporaryDirectory

In [ ]:
# | export


@contextmanager
def set_cwd(cwd_path: Union[Path, str]):

    cwd_path = Path(cwd_path)
    original_cwd = os.getcwd()
    os.chdir(cwd_path)

    try:
        nbdev.config.get_config.cache_clear()
        yield
    finally:
        os.chdir(original_cwd)


In [ ]:
with TemporaryDirectory() as d:
    with set_cwd(d):
        assert (
            Path(os.getcwd()) == Path(d).resolve()
        ), f"{os.getcwd()}, {Path(d).resolve()}"


In [ ]:
# | export


def get_value_from_config(root_path: str, config_name: str) -> str:
    """Get the value from settings.ini file"""

    settings_path = Path(root_path) / "settings.ini"
    config = ConfigParser()
    config.read(settings_path)
    if not config.has_option("DEFAULT", config_name):
        return ""
    return config["DEFAULT"][config_name]


In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)
    ret_val = get_value_from_config(d, "lib_path")
    print(ret_val)
    assert ret_val == "nbdev_mkdocs", ret_val
    
    ret_val = get_value_from_config(d, "repo")
    print(ret_val)
    assert ret_val == "nbdev-mkdocs", ret_val
    
    ret_val = get_value_from_config(d, "user")
    print(ret_val)
    assert ret_val == "airtai", ret_val


nbdev_mkdocs
nbdev-mkdocs
airtai
